In [158]:
from bigtree import Node
from bigtree import dict_to_tree
from bigtree import dataframe_to_tree
import numpy as np
import pandas as pd
import json

In [159]:
df1 = pd.read_csv('C:/Users/brian/Documents/data-transformers/tests/data/two_tables/df_topic.csv')
df2 = pd.read_csv("C:/Users/brian/Documents/data-transformers/tests/data/two_tables/df_doc.csv")
schema = pd.read_csv('C:/Users/brian/Documents/data-transformers/tests/data/two_tables/postgres_information_schema.csv')

#### To-Do list: 

1. figure out how to identify the dataframe that has the foreign key 
    - possible methods:
        - adding titles to each dataframe
        - conrelid and confrelid say the csv file names instead and that could be used

2. if the foreign key names ('attname') are different, then the code needs to be updated to reflect that case scenario
    - current fix:
        - renaming both attname to be the same in each dataframe (it's a quick fix but that does mess up the actual schema so idk if that's worth it)



In [161]:
#code: table with foreign key has to be df1

In [162]:
def two_tables(df1, df2, schema):
    df1_name = schema['conrelid'][0] #name of table the constraint is on, foreign key
    df2_name = schema['confrelid'][0] #name of refrenced table
    
    shared_column = schema['attname'][0] #name of the foreign key in one table, that is connected with a corresponding column from the refrenced table.
    if shared_column != schema['attname.1'][0]: 
        df2 = df2.rename(columns={schema['attname.1'][0]: shared_column}) #if the attnames are different, we will rename them to be the same 
    
    df1_columns = list(np.setdiff1d(df1.columns, [shared_column])) #columns in table besides shared column
    df2_columns = list(np.setdiff1d(df2.columns, [shared_column]))
    
    dup_col_name = np.intersect1d(df2_columns, df1_columns) #if both dataframes have columns with the same name that's not a foreign key
    if len(dup_col_name) != 0:
        for col in dup_col_name:
            df1 = df1.rename({col: col+'_x'}, axis=1) #renaming both columns
            df2 = df2.rename({col: col+'_y'}, axis=1)
            df1_columns = list(np.setdiff1d(df1.columns, [shared_column])) 
            df2_columns = list(np.setdiff1d(df2.columns, [shared_column]))
        
    df = df1.merge(df2, on=shared_column)  
    all_cols = [shared_column] + df2_columns 
    temp = (df.groupby(all_cols)[df1_columns]
           .apply(lambda x: x.to_dict('records'))
           .reset_index(name = df1_name)
           .to_dict(orient='records')
          )

    final = json.dumps({df1_name:temp}, indent = 4)
    print(final)

In [163]:
two_tables(df1, df2, schema)

{
    "topics": [
        {
            "document_id": 1,
            "name_y": "aaa",
            "topics": [
                {
                    "name_x": "xxx",
                    "topic_id": 1
                }
            ]
        },
        {
            "document_id": 2,
            "name_y": "bbb",
            "topics": [
                {
                    "name_x": "yyy",
                    "topic_id": 2
                },
                {
                    "name_x": "zzz",
                    "topic_id": 3
                }
            ]
        }
    ]
}


In [164]:
df1 = pd.DataFrame([(1, 'August', 3000),
                    (1, 'September', 3200),
                    (1, 'October', 3400),
                    (2, 'August', 2000),
                    (2, 'September', 2200),
                    (2, 'October', 2400),
                    (3, 'August', 1000),
                    (3, 'September', 2400),
                    (3, 'October', 3500),
                    (4, 'August', 4000),
                    (4, 'September', 3600),
                    (4, 'October', 2300)],
                    columns = ['Emp_id', 'Month', 'Income'])

df2 = pd.DataFrame([(1, 'M', 23),
                    (2, 'F', 29),
                    (3, 'M', 31),
                    (4, 'F', 38)],
                  columns = ['Emp_id', 'Gender', 'Age'])

schema = pd.DataFrame([('employee_fk_payroll', 'payroll', 'Emp_id', 'employee', 'Emp_id')],
                     columns = ['conname', 'conrelid', 'attname', 'confrelid', 'attname.1'])

In [165]:
two_tables(df1, df2, schema)

{
    "payroll": [
        {
            "Emp_id": 1,
            "Age": 23,
            "Gender": "M",
            "payroll": [
                {
                    "Income": 3000,
                    "Month": "August"
                },
                {
                    "Income": 3200,
                    "Month": "September"
                },
                {
                    "Income": 3400,
                    "Month": "October"
                }
            ]
        },
        {
            "Emp_id": 2,
            "Age": 29,
            "Gender": "F",
            "payroll": [
                {
                    "Income": 2000,
                    "Month": "August"
                },
                {
                    "Income": 2200,
                    "Month": "September"
                },
                {
                    "Income": 2400,
                    "Month": "October"
                }
            ]
        },
        {
            "Emp_id": 3,
    

In this scenario, the foreign key name is different from the corresponding name of the refrenced column

In [167]:
df1 = pd.DataFrame([(1, 'August', 3000),
                    (1, 'September', 3200),
                    (1, 'October', 3400),
                    (2, 'August', 2000),
                    (2, 'September', 2200),
                    (2, 'October', 2400),
                    (3, 'August', 1000),
                    (3, 'September', 2400),
                    (3, 'October', 3500),
                    (4, 'August', 4000),
                    (4, 'September', 3600),
                    (4, 'October', 2300)],
                    columns = ['Emp_id', 'Month', 'Income'])

df2 = pd.DataFrame([(1, 'M', 23),
                    (2, 'F', 29),
                    (3, 'M', 31),
                    (4, 'F', 38)],
                  columns = ['Id', 'Gender', 'Age'])

schema = pd.DataFrame([('employee_fk_payroll', 'payroll', 'Emp_id', 'employee', 'Id')],
                     columns = ['conname', 'conrelid', 'attname', 'confrelid', 'attname.1'])


In [173]:
two_tables(df1, df2, schema)

{
    "payroll": [
        {
            "Emp_id": 1,
            "Age": 23,
            "Gender": "M",
            "payroll": [
                {
                    "Income": 3000,
                    "Month": "August"
                },
                {
                    "Income": 3200,
                    "Month": "September"
                },
                {
                    "Income": 3400,
                    "Month": "October"
                }
            ]
        },
        {
            "Emp_id": 2,
            "Age": 29,
            "Gender": "F",
            "payroll": [
                {
                    "Income": 2000,
                    "Month": "August"
                },
                {
                    "Income": 2200,
                    "Month": "September"
                },
                {
                    "Income": 2400,
                    "Month": "October"
                }
            ]
        },
        {
            "Emp_id": 3,
    

In [168]:
df1_name = schema['conrelid'][0] #name of table the constraint is on, foreign key
df2_name = schema['confrelid'][0] #name of refrenced table

In [169]:
shared_column = schema['attname'][0] #name of the foreign key in one table, that is connected with a corresponding column from the refrenced table.
if shared_column != schema['attname.1'][0]:
    df2 = df2.rename(columns={schema['attname.1'][0]: shared_column})
    

In [170]:
df1_columns = list(np.setdiff1d(df1.columns, [shared_column])) 
df2_columns = list(np.setdiff1d(df2.columns, [shared_column]))

In [171]:
dup_col_name = np.intersect1d(df2_columns, df1_columns) #if both dataframes have a column with the same name that's not a foreign key
if len(dup_col_name) != 0:
    for col in dup_col_name:
        df1 = df1.rename({col: col+'_x'}, axis=1) #renaming both columns
        df2 = df2.rename({col: col+'_y'}, axis=1)
        df1_columns = list(np.setdiff1d(df1.columns, [shared_column])) 
        df2_columns = list(np.setdiff1d(df2.columns, [shared_column]))

In [172]:
df = df1.merge(df2, on=shared_column)
all_cols = [shared_column] + df2_columns
temp = (df.groupby(all_cols)[df1_columns]
       .apply(lambda x: x.to_dict('records'))
       .reset_index(name = df1_name)
       .to_dict(orient='records')
      )

final = json.dumps({df1_name:temp}, indent = 4)
print(final)

{
    "payroll": [
        {
            "Emp_id": 1,
            "Age": 23,
            "Gender": "M",
            "payroll": [
                {
                    "Income": 3000,
                    "Month": "August"
                },
                {
                    "Income": 3200,
                    "Month": "September"
                },
                {
                    "Income": 3400,
                    "Month": "October"
                }
            ]
        },
        {
            "Emp_id": 2,
            "Age": 29,
            "Gender": "F",
            "payroll": [
                {
                    "Income": 2000,
                    "Month": "August"
                },
                {
                    "Income": 2200,
                    "Month": "September"
                },
                {
                    "Income": 2400,
                    "Month": "October"
                }
            ]
        },
        {
            "Emp_id": 3,
    

## One Relational Table to Semi-Structured (Tree/JSON)

In [212]:
base_url = "https://github.com/byuidatascience/data4python4ds/raw/master/data-raw/"
file = "{}flights/flights.csv".format(base_url)

flights = pd.read_csv(file)
weather = pd.read_csv("{}weather/weather.csv".format(base_url))
airlines = pd.read_csv("{}airlines/airlines.csv".format(base_url))

In [213]:
#getting the file name from the file path 
file_name = file.split('/')[-1].split('.')[0]
file_name

'flights'

In [214]:
#sample of one relational table
df = flights.head(20)
df

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z
5,2013,1,1,554.0,558,-4.0,740.0,728,12.0,UA,1696,N39463,EWR,ORD,150.0,719,5,58,2013-01-01T10:00:00Z
6,2013,1,1,555.0,600,-5.0,913.0,854,19.0,B6,507,N516JB,EWR,FLL,158.0,1065,6,0,2013-01-01T11:00:00Z
7,2013,1,1,557.0,600,-3.0,709.0,723,-14.0,EV,5708,N829AS,LGA,IAD,53.0,229,6,0,2013-01-01T11:00:00Z
8,2013,1,1,557.0,600,-3.0,838.0,846,-8.0,B6,79,N593JB,JFK,MCO,140.0,944,6,0,2013-01-01T11:00:00Z
9,2013,1,1,558.0,600,-2.0,753.0,745,8.0,AA,301,N3ALAA,LGA,ORD,138.0,733,6,0,2013-01-01T11:00:00Z


In [215]:
#turns the relational table into JSON Tree
results = df.to_json(orient='records')
parsed = json.loads(results)
print(json.dumps({file_name: parsed}, indent=4))

{
    "flights": [
        {
            "year": 2013,
            "month": 1,
            "day": 1,
            "dep_time": 517.0,
            "sched_dep_time": 515,
            "dep_delay": 2.0,
            "arr_time": 830.0,
            "sched_arr_time": 819,
            "arr_delay": 11.0,
            "carrier": "UA",
            "flight": 1545,
            "tailnum": "N14228",
            "origin": "EWR",
            "dest": "IAH",
            "air_time": 227.0,
            "distance": 1400,
            "hour": 5,
            "minute": 15,
            "time_hour": "2013-01-01T10:00:00Z"
        },
        {
            "year": 2013,
            "month": 1,
            "day": 1,
            "dep_time": 533.0,
            "sched_dep_time": 529,
            "dep_delay": 4.0,
            "arr_time": 850.0,
            "sched_arr_time": 830,
            "arr_delay": 20.0,
            "carrier": "UA",
            "flight": 1714,
            "tailnum": "N24211",
            "origin": "LGA

## Two Relational Tables to Semi-Structured (Tree/JSON)

In [216]:
df1 = pd.DataFrame([(1, 'M', 23),
                    (2, 'F', 29),
                    (3, 'M', 31),
                    (4, 'F', 38)],
                  columns = ['Emp_id', 'Gender', 'Age'])

df2 = pd.DataFrame([(1, 'August', 3000),
                    (1, 'September', 3200),
                    (1, 'October', 3400),
                    (2, 'August', 2000),
                    (2, 'September', 2200),
                    (2, 'October', 2400),
                    (3, 'August', 1000),
                    (3, 'September', 2400),
                    (3, 'October', 3500),
                    (4, 'August', 4000),
                    (4, 'September', 3600),
                    (4, 'October', 2300)],
                    columns = ['Emp_id', 'Month', 'Income'])


df1_name = 'Employee'
df2_name = 'Payroll'

In [217]:
df1

,Emp_id,Gender,Age
0,1,M,23
1,2,F,29
2,3,M,31
3,4,F,38


In [218]:
df2

,Emp_id,Month,Income
0,1,August,3000
1,1,September,3200
2,1,October,3400
3,2,August,2000
4,2,September,2200
5,2,October,2400
6,3,August,1000
7,3,September,2400
8,3,October,3500
9,4,August,4000


In [219]:
#merging the two relational tables by a common column
temp = df1.merge(df2, on='Emp_id')
cols = temp.columns.difference(['Emp_id', 'Gender', 'Age'])[::-1]
temp

,Emp_id,Gender,Age,Month,Income
0,1,M,23,August,3000
1,1,M,23,September,3200
2,1,M,23,October,3400
3,2,F,29,August,2000
4,2,F,29,September,2200
5,2,F,29,October,2400
6,3,M,31,August,1000
7,3,M,31,September,2400
8,3,M,31,October,3500
9,4,F,38,August,4000


In [220]:
results = (temp.groupby(['Emp_id', 'Gender', 'Age'])[cols]
         .apply(lambda x: [dict(x.values)])
         .reset_index(name = df2_name)
         .to_json(orient='records')
        )
parsed = json.loads(results)
print(json.dumps({df1_name: parsed}, indent=4))

{
    "Employee": [
        {
            "Emp_id": 1,
            "Gender": "M",
            "Age": 23,
            "Payroll": [
                {
                    "August": 3000,
                    "September": 3200,
                    "October": 3400
                }
            ]
        },
        {
            "Emp_id": 2,
            "Gender": "F",
            "Age": 29,
            "Payroll": [
                {
                    "August": 2000,
                    "September": 2200,
                    "October": 2400
                }
            ]
        },
        {
            "Emp_id": 3,
            "Gender": "M",
            "Age": 31,
            "Payroll": [
                {
                    "August": 1000,
                    "September": 2400,
                    "October": 3500
                }
            ]
        },
        {
            "Emp_id": 4,
            "Gender": "F",
            "Age": 38,
            "Payroll": [
                {
      

### Trying to generalize code so any two relational tables can be transformed into a nested Json Tree

In [221]:
shared_column = np.intersect1d(df2.columns, df1.columns)[0] 
df1_columns = list(np.setdiff1d(df1.columns, df2.columns))
df2_columns = list(np.setdiff1d(df2.columns, df1.columns))[::-1]

all_cols = [shared_column] + df1_columns
temp = (df1.merge(df2, on=shared_column)
       .groupby(all_cols)[df2_columns]
       .apply(lambda x: [dict(x.values)])
       .reset_index(name = df2_name)
       .to_dict(orient='records')
      )
final = json.dumps({df1_name:temp}, indent = 4)
print(final)

{
    "Employee": [
        {
            "Emp_id": 1,
            "Age": 23,
            "Gender": "M",
            "Payroll": [
                {
                    "August": 3000,
                    "September": 3200,
                    "October": 3400
                }
            ]
        },
        {
            "Emp_id": 2,
            "Age": 29,
            "Gender": "F",
            "Payroll": [
                {
                    "August": 2000,
                    "September": 2200,
                    "October": 2400
                }
            ]
        },
        {
            "Emp_id": 3,
            "Age": 31,
            "Gender": "M",
            "Payroll": [
                {
                    "August": 1000,
                    "September": 2400,
                    "October": 3500
                }
            ]
        },
        {
            "Emp_id": 4,
            "Age": 38,
            "Gender": "F",
            "Payroll": [
                {
      

In [197]:
shared_column = np.intersect1d(df2.columns, df1.columns)[0] 
df1_columns = list(np.setdiff1d(df1.columns, df2.columns))
df2_columns = list(np.setdiff1d(df2.columns, df1.columns))[::-1]

all_cols = [shared_column] + df1_columns
temp = (df1.merge(df2, on=shared_column)
       .groupby(all_cols)[df2_columns]
       .apply(lambda x: x.to_dict())
       .reset_index(name = df2_name)
       .to_dict(orient='records')
      )
final = json.dumps({df1_name:temp}, indent = 4)
print(final)

{
    "flights": [
        {
            "Emp_id": 1,
            "Age": 23,
            "Gender": "M",
            "df2": {
                "Month": {
                    "0": "August",
                    "1": "September",
                    "2": "October"
                },
                "Income": {
                    "0": 3000,
                    "1": 3200,
                    "2": 3400
                }
            }
        },
        {
            "Emp_id": 2,
            "Age": 29,
            "Gender": "F",
            "df2": {
                "Month": {
                    "3": "August",
                    "4": "September",
                    "5": "October"
                },
                "Income": {
                    "3": 2000,
                    "4": 2200,
                    "5": 2400
                }
            }
        },
        {
            "Emp_id": 3,
            "Age": 31,
            "Gender": "M",
            "df2": {
                "Month": {
  

In [202]:
df1 = pd.DataFrame([(1, 'Kolter', 'Zico', 1), 
                   (2, 'Xi', 'Edgar', 2),
                   (3, 'Lee', 'Mark', 2), 
                   (4, 'Mani', 'Shouvik', 2),
                   (5, 'Gates', 'Bill', 3),
                   (6, 'Musk', 'Elon', 3)], 
                  columns=["id", "last_name", "first_name","role_id"])

df2 = pd.DataFrame([(1, "Instructor"),
                    (2, "TA"),
                    (3, "Student")],
                  columns = ["role_id", "title"])

df1_name = 'Person'
df2_name = 'Role'

In [204]:
shared_column = np.intersect1d(df2.columns, df1.columns)[0] 
df1_columns = list(np.setdiff1d(df1.columns, df2.columns))
df2_columns = list(np.setdiff1d(df2.columns, df1.columns))[::-1]

all_cols = [shared_column] + df1_columns
temp = (df1.merge(df2, on=shared_column)
       .groupby(all_cols)[df2_columns]
       .apply(lambda x: x.to_dict())
       .reset_index(name = df2_name)
       .to_dict(orient='records')
      )
final = json.dumps({df1_name:temp}, indent = 4)
print(final)

{
    "Person": [
        {
            "role_id": 1,
            "first_name": "Zico",
            "id": 1,
            "last_name": "Kolter",
            "Role": {
                "title": {
                    "0": "Instructor"
                }
            }
        },
        {
            "role_id": 2,
            "first_name": "Edgar",
            "id": 2,
            "last_name": "Xi",
            "Role": {
                "title": {
                    "1": "TA"
                }
            }
        },
        {
            "role_id": 2,
            "first_name": "Mark",
            "id": 3,
            "last_name": "Lee",
            "Role": {
                "title": {
                    "2": "TA"
                }
            }
        },
        {
            "role_id": 2,
            "first_name": "Shouvik",
            "id": 4,
            "last_name": "Mani",
            "Role": {
                "title": {
                    "3": "TA"
                }
            